# Multiheadattention 이해 요함.
# keras vs torch
# permute 왜 했는지 이해해야함.
# Layer norm ; keras vs torch
-> layer norm 자체도 디테일하게 파라미터를 전부 공유하는지등 이해해야함.
- tf.keras.layers.LayerNormalization tensorflow에 이거 제공함.

# 일단 tensorflow에 동일한 built in method들이 존재해서 그대로 결과를 볼 수 있을 것 같음.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
import bottleneck as bn
import keras
import math

In [ ]:
import argparse
import pickle
import time
import numpy as np

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master/

/content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master


In [ ]:
train_data = pickle.load(open('../datasets/' + 'diginetica' + '/train.txt', 'rb'))

In [ ]:

def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        '''
        print("##### in get_slice, inputs")
        print(inputs)
        print(len(inputs),len(inputs[0]),len(inputs[1]))
        print("##### in get_slice, mask")
        print(mask)
        print("##### in get_slice, targets")
        print(targets)
        '''
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input))) 
        max_n_node = np.max(n_node) # 해당 slices 중 한 sequence 안에서 가질 수 있는 아이템의 종류의 최댓값
        # 0 패딩 후에 개수를 따지는 것이기 때문에 id 개수 +1 임.
        
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node)) # 균일한 크기로 생성.
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets


In [ ]:
train_data, valid_data = split_validation(train_data, 0.1)
test_data = valid_data

In [ ]:
train_data1 = Data(train_data, shuffle=False)
test_data1 = Data(test_data, shuffle=False)

In [ ]:
class opt:
  def __init__(self,step,hidden_size,batch_size,nonhybrid=None):
    self.step=step
    self.hiddenSize = hidden_size
    #self.n_node = n_node
    self.batchSize = batch_size
    self.nonhybrid = nonhybrid
    self.lr = 0.001
    self.l2 = 1e-5
    self.lr_dc_step = 3
    self.lr_dc = 0.1
    self.epoch = 30
    self.patience = 10
    

In [ ]:
oopt=opt(step = 1,hidden_size=100,batch_size=100)

In [ ]:
n_node = 43098

In [ ]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        # self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.stdv = 1/math.sqrt(self.hidden_size)
        self.w_ih = self.add_weight(shape=(self.input_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        AttributeError: module 'keras.api._v2.keras.initializers' has no attribute 'Uniform'
        add_weight doesn't have the attribute named 'kernel_initializer'
        '''
        # self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.w_hh = self.add_weight(shape=(self.hidden_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_ih = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        
        # self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
                
        # self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_iah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_oah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_in = tf.keras.layers.Dense(self.hidden_size, 
                                                    use_bias=True,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        TypeError: ('Unknown keyword argument:', 'bias') - > use_bias=True in Dense
        '''
        # self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

    def Cell(self, A, hidden):
        input_in = tf.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = tf.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]],self.linear_edge_out(hidden)) + self.b_oah
        inputs = tf.concat([input_in, input_out], 2) 
        gi = tf.matmul(inputs, self.w_ih )+ self.b_ih # Dense layer와 같은 역할함.
        # gate_size가 앞서서 3* hidden_size로 정의되어 있었고 아래 chunk에서 split하는 것 같다.
        gh = tf.matmul(hidden, self.w_hh) + self.b_hh
        i_r, i_i, i_n = tf.split(gi,3,axis = 2)
        h_r, h_i, h_n = tf.split(gh,3,axis = 2)
        resetgate = tf.nn.sigmoid(i_r + h_r)
        inputgate = tf.nn.sigmoid(i_i + h_i)
        newgate = tf.nn.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def call(self, A, hidden):
        for i in range(self.step):
            hidden = self.Cell(A, hidden)
        return hidden

In [ ]:

class TAGNN(tf.keras.models.Model):
    def __init__(self, opt, n_node):
        super(TAGNN, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.nonhybrid = opt.nonhybrid
        # self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.embedding = tf.keras.layers.Embedding(input_dim = self.n_node,output_dim=self.hidden_size)
        self.stdv = 1/math.sqrt(self.hidden_size)
        print(self.stdv)
        self.gnn = GNN(self.hidden_size, step=opt.step)

        # self.layer_norm1 = nn.LayerNorm(self.hidden_size)
        self.layer_norm1 =tf.keras.layers.LayerNormalization(axis = -1)
        # self.attn = tf.keras.layers.MultiheadAttention( embed_dim=self.hidden_size, num_heads=2, dropout=0.1)
        self.attn = tf.keras.layers.MultiHeadAttention(num_heads=2,key_dim=self.hidden_size,dropout=0.1)

        # self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_one = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_three = tf.keras.layers.Dense(1 , use_bias=False,
                                                  kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                  )
        # self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        
        self.linear_transform = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                      kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                      bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_t = nn.Linear(self.hidden_size, self.hidden_size, bias=False)  #target attention
        self.linear_t = tf.keras.layers.Dense( self.hidden_size, use_bias=False,
                                              kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                              )

    def call(self, inputs, A, mask,alias_inputs):
        hidden = self.embedding(inputs) 
        hidden = self.gnn(A, hidden)

        hidden = tf.transpose(hidden,[1,0,2])
        skip = self.layer_norm1(hidden)


        hidden, attn_w = self.attn(
            key = hidden, query = hidden, value = hidden,return_attention_scores=True)
            # hidden, hidden, hidden, attn_mask=get_mask(hidden.shape[0])) 원래 이런식으로 mask까지 해주어야 하는데
            # torch와 tensorflow mask 방식이 좀 달라서 공부 필요.
        hidden = hidden+skip
        hidden = tf.transpose(hidden,[1,0,2])
        
        get = lambda j: tf.gather(hidden[j],alias_inputs[j])
        hidden = tf.stack([get(j) for j in range(len(alias_inputs))])
        ht = tf.gather_nd(hidden ,indices = tf.stack([tf.range(mask.shape[0],dtype='int64') , tf.cast(tf.math.reduce_sum(mask,1),'int64') -1],axis=1))
        # tf.reshape을 이용해야함.
        q1 = tf.reshape(self.linear_one(ht) ,[ht.shape[0], 1, ht.shape[1]])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(tf.nn.sigmoid(q1 + q2))  # (b,s,1)
        
        alpha = tf.nn.softmax(alpha, 1) # B,S,1
        
        
        a = tf.math.reduce_sum(alpha * hidden * tf.cast(tf.reshape(mask, [mask.shape[0], -1, 1]),dtype = 'float32'), 1)  # (b,d)
        if not self.nonhybrid: # 여기 적용되는지 판단할 것
        
            a = self.linear_transform(tf.concat([a, ht], 1)) 
        
        b = self.embedding.weights[0][1:] # 이 차원이 맞긴 함.

        hidden = hidden * tf.cast(tf.reshape(mask,[mask.shape[0], -1, 1]),'float32')  # batch_size x seq_length x latent_size
        

        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size

        beta = tf.nn.softmax(b @ tf.transpose(qt,[0,2,1]), -1)  # batch_size x n_nodes x seq_length
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = tf.reshape(a,[ht.shape[0], 1, ht.shape[1]])  # b,1,d
        a = a + target  # b,n,d # 이거 concat인가 ? 

        scores = tf.math.reduce_sum(a * b, -1)  # b,n
        scores=tf.nn.softmax(scores,-1)
        return scores

In [ ]:
model=TAGNN(oopt,n_node)

loss_ftn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = oopt.lr)


#@tf.function
def train_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    with tf.GradientTape() as tape :
      scores = model(item_input, A_input, mask_input,alias_inputs_,training =True)
      
      ll = loss_ftn(targets_input-1,scores)

    
    
    gradients =tape.gradient(ll,model.trainable_weights)

    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    return ll

0.1


In [ ]:
%load_ext tensorboard

In [ ]:
import datetime

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
train_log_dir = './' + current_time + '/train'
tb_callback = tf.keras.callbacks.TensorBoard(train_log_dir)
tb_callback.set_model(model)

In [ ]:
for epoch in range(oopt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    slices = train_data1.generate_batch(model.batch_size)
    loss = 0
    for i, j in zip(slices, np.arange(len(slices))):
        alias_inputs, A, items, mask, targets = train_data1.get_slice(i)

        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)
        
        l = train_ftn(items,A,mask,alias_inputs,targets)
        
        loss += l 
        
        if j % int(100) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), l))
    print('\tLoss:\t%.3f' % loss)
    hit, mrr = [], []
    slices = test_data1.generate_batch(model.batch_size)
    for i in slices:
        alias_inputs, A, items, mask, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100

    print(f"Recall@20 : {hit} , MRR@20 {mrr}")

Recall@20 : 51.51152932019403 , MRR@20 17.93724189078418

Recall@20 : 52.692954535977876 , MRR@20 18.589936593912345

Recall@20 : 50.899968032023565 , MRR@20 18.03496322533131

Recall@20 : 48.58020487303153 , MRR@20 17.22774014883739

- 에포크 4번만에 떨어지는 모습.